# Gaussova eliminacija

## Općenito

Sustav $Ax=b$ se rješava u tri koraka (__bez pivotiranja__)

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija)
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija)
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija)

Ukoliko pivotiramo, tada je

1. $PA=LU$ 
2. $Ly=P^T b$
3. $Ux=y$ 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.928851  0.826131   0.179541   0.193666  0.970172   0.32579 
 0.895041  0.103842   0.856939   0.76241   0.70185    0.725512
 0.825504  0.141956   0.994111   0.444585  0.665407   0.201752
 0.969686  0.0287001  0.0771153  0.995251  0.0924995  0.98151 
 0.34744   0.287551   0.137898   0.451277  0.884962   0.171592
 0.154071  0.705012   0.725469   0.662972  0.786284   0.490006

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
 1.0        0.0         0.0        0.0        0.0      0.0
 0.9636     1.0         0.0        0.0        0.0      0.0
 0.888736   0.855592    1.0        0.0        0.0      0.0
 1.04396    1.20446    -3.74566    1.0        0.0      0.0
 0.374053   0.0310106   0.198619  -0.558099   1.0      0.0
 0.165873  -0.82052     5.04      -3.05173   -8.47973  1.0,

6x6 Array{Float64,2}:
 0.928851   0.826131  0.179541   0.193666   0.970172     0.32579 
 0.0       -0.692218  0.683933   0.575793  -0.233008     0.411581
 0.0        0.0       0.249379  -0.220178   0.00253942  -0.439935
 0.0        0.0       0.0       -0.725161  -0.630163    -1.50218 
 0.0        0.0       0.0        0.0        0.177094    -0.714022
 0.0        0.0       0.0        0.0        0.0         -7.64802 )

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0  0.0  0.0   0.0          0.0          0.0
 0.0  0.0  0.0   0.0          0.0          0.0
 0.0  0.0  0.0   5.55112e-17  0.0          0.0
 0.0  0.0  0.0   0.0          0.0          0.0
 0.0  0.0  0.0  -5.55112e-17  0.0          0.0
 0.0  0.0  0.0   0.0          4.44089e-16  0.0

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.398146
 0.915821
 0.446334
 0.67038 
 0.933759
 0.941788

In [8]:
x=A\b

6-element Array{Float64,1}:
 -0.479773
 -0.440174
 -0.131435
  0.643558
  0.994701
  0.433889

In [9]:
y=myL(L,b)

6-element Array{Float64,1}:
  0.398146
  0.532168
 -0.362831
 -1.74529 
 -0.133651
 -3.31839 

In [10]:
x1=myU(U,y)

6-element Array{Float64,1}:
 -0.479773
 -0.440174
 -0.131435
  0.643558
  0.994701
  0.433889

In [11]:
x-x1

6-element Array{Float64,1}:
 -1.11022e-16
  5.55112e-16
  8.32667e-17
  6.66134e-16
 -1.11022e-16
 -5.55112e-16

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1.

In [12]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [13]:
mylu1(A)

6x6 Array{Float64,2}:
 0.928851   0.826131    0.179541   0.193666   0.970172     0.32579 
 0.9636    -0.692218    0.683933   0.575793  -0.233008     0.411581
 0.888736   0.855592    0.249379  -0.220178   0.00253942  -0.439935
 1.04396    1.20446    -3.74566   -0.725161  -0.630163    -1.50218 
 0.374053   0.0310106   0.198619  -0.558099   0.177094    -0.714022
 0.165873  -0.82052     5.04      -3.05173   -8.47973     -7.64802 

In [14]:
L,U

(
6x6 Array{Float64,2}:
 1.0        0.0         0.0        0.0        0.0      0.0
 0.9636     1.0         0.0        0.0        0.0      0.0
 0.888736   0.855592    1.0        0.0        0.0      0.0
 1.04396    1.20446    -3.74566    1.0        0.0      0.0
 0.374053   0.0310106   0.198619  -0.558099   1.0      0.0
 0.165873  -0.82052     5.04      -3.05173   -8.47973  1.0,

6x6 Array{Float64,2}:
 0.928851   0.826131  0.179541   0.193666   0.970172     0.32579 
 0.0       -0.692218  0.683933   0.575793  -0.233008     0.411581
 0.0        0.0       0.249379  -0.220178   0.00253942  -0.439935
 0.0        0.0       0.0       -0.725161  -0.630163    -1.50218 
 0.0        0.0       0.0        0.0        0.177094    -0.714022
 0.0        0.0       0.0        0.0        0.0         -7.64802 )

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [15]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.902792    0.0529483   0.114961     …  0.189585   0.331006  0.0942038 
 0.459294    0.753521    0.602996        0.231372   0.927549  0.12609   
 0.972858    0.404017    0.156277        0.626693   0.205268  0.95323   
 0.762735    0.362801    0.892138        0.287035   0.782965  0.79254   
 0.490025    0.542825    0.810292        0.603035   0.719931  0.00376585
 0.998519    0.167613    0.930185     …  0.702441   0.676787  0.211241  
 0.914578    0.506561    0.522011        0.404128   0.974807  0.508973  
 0.594145    0.171204    0.484604        0.55249    0.60707   0.104811  
 0.729423    0.976428    0.000842566     0.0450883  0.479209  0.418361  
 0.213938    0.697895    0.339668        0.490263   0.609231  0.946934  
 0.465744    0.23645     0.437169     …  0.446787   0.50403   0.340749  
 0.716006    0.837383    0.0900643       0.877639   0.933053  0.440147  
 0.792665    0.833628    0.986618        0.55228    0.513088  0.160584  
 ⋮                       

In [17]:
@time lu(A);

  0.159373 seconds (55 allocations: 6.010 MB)


In [19]:
@time mylu1(A);

  0.387017 seconds (17.26 k allocations: 1.004 GB, 7.93% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji. `mylu1()` je malo sporiji.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [21]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [22]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [23]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.417082  0.165465   0.390313  0.13527 
 0.82179   0.902387   0.809553  0.345416
 0.429799  0.872422   0.181622  0.165782
 0.742942  0.0559246  0.449096  0.339387  …  4x4 Array{Float64,2}:
 0.615058  0.0748728  0.499947  0.332858
 0.799362  0.0237195  0.314149  0.119406
 0.578529  0.597041   0.362772  0.141244
 0.627861  0.97874    0.966568  0.751086                            
 4x4 Array{Float64,2}:
 -9.23696   3.60158   -1.7303    2.74318
 -9.03583   3.08368   -1.6778    3.39122
 -7.31174   3.22409   -2.05427   2.85337
 -2.15468  -0.184459   0.652457  2.0528      4x4 Array{Float64,2}:
 2.72953   -0.960275   1.58832    1.24685 
 2.54755   -1.70329    1.68185    0.489873
 1.71969   -0.919584   1.58836    1.16362 
 0.207942  -1.96691   -0.532115  -0.247906                    
 4x4 Array{Float64,2}:
 -12.5264  5.06691  -2.05647  2.8186 
 -17.1998  6.67422  -3.8237   4.64157
 -10.6573  4.86332  -1.99745  1.85448
 -13.5718  6.5743   -3.81032  3.397

In [24]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [25]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                                  …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                        
 4x4 Array{Float64,2}:
  2.22045e-16   1.11022e-16   2.22045e-16   0.0        
 -6.66134e-16  -5.55112e-17  -6.66134e-16  -4.44089e-16
  0.0          -2.77556e-16  -4.44089e-16  -2.22045e-16
  1.11022e-16   1.11022e-16   2.22045e-16  -5.55112e-17     4x4 Array{Float64,2}:
 2.22045e-16  0.0  0.0  0.0
 0.0          0.0  0.0  0.0
 0.0          0.0  0.0  0.0
 0.0          0.0  0.0  0.0                                                                                                        
 4x4

In [26]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [27]:
norm(unblock(Res))

6.001655326510322e-15

Sada probajmo veću dimenziju:

In [28]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [31]:
@time mylu2(Ab);

  0.166623 seconds (8.79 k allocations: 26.526 MB, 0.94% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> što u praksi dovoljno spriječava rast elemenata.


In [32]:
A=rand(5,5)
L,U,P=lu(A)

(
5x5 Array{Float64,2}:
 1.0       0.0       0.0        0.0        0.0
 0.14147   1.0       0.0        0.0        0.0
 0.178195  0.421298  1.0        0.0        0.0
 0.462087  0.390092  0.148616   1.0        0.0
 0.57052   0.890231  0.50088   -0.0497234  1.0,

5x5 Array{Float64,2}:
 0.814091  0.380744  0.491771   0.595496   0.412362
 0.0       0.48304   0.302009   0.72673    0.455245
 0.0       0.0       0.708493  -0.256694   0.196174
 0.0       0.0       0.0        0.311776   0.183652
 0.0       0.0       0.0        0.0       -0.561589,

[2,1,5,3,4])

In [33]:
L*U-A[P,:]

5x5 Array{Float64,2}:
 0.0   0.0          0.0  0.0          0.0
 0.0   0.0          0.0  0.0          0.0
 0.0   0.0          0.0  0.0          0.0
 0.0   0.0          0.0  1.11022e-16  0.0
 0.0  -1.11022e-16  0.0  0.0          0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [34]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [35]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.534202
 0.682644
 0.71686 
 0.473825
 0.919802

In [36]:
L,U,Pr,Pc=gecp(A)

(
5x5 Array{Float64,2}:
 1.0         0.0        0.0        0.0       0.0
 0.0911512   1.0        0.0        0.0       0.0
 0.682288    0.768991   1.0        0.0       0.0
 0.78931    -0.257312  -0.11616    1.0       0.0
 0.767545    0.312559   0.318382  -0.928332  1.0,

5x5 Array{Float64,2}:
 0.975517  0.502101   0.657609   0.689226   0.534002 
 0.0       0.816636   0.730769   0.309203   0.766391 
 0.0       0.0       -0.877071   0.114622  -0.793538 
 0.0       0.0        0.0       -0.324985  -0.0852189
 0.0       0.0        0.0        0.0        0.0105081,

5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0,

5x5 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0)

In [37]:
Pr*L*U*Pc'-A

5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  0.0          -5.55112e-17
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  1.11022e-16   0.0        

In [38]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
 0.534202
 0.425132
 0.2284  
 0.431132
 0.467256

In [39]:
z=myU(U,y)

5-element Array{Float64,1}:
  13.0706 
   1.46023
 -42.189  
 -12.9867 
  44.4664 

In [40]:
x=Pc*z

5-element Array{Float64,1}:
 -12.9867 
   1.46023
  13.0706 
 -42.189  
  44.4664 

In [41]:
A*x-b

5-element Array{Float64,1}:
 -4.44089e-16
  8.88178e-16
  0.0        
  7.54952e-15
 -2.22045e-15

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$
za neki $\hat x=x+\delta x$.

Želimo ocijeniti 
$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 87, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$
odnosno
$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$
odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$ imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je _uvjetovanost_ (_kondicija_)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [R. Scitovski, Numerička matematika, str. 42][RS04]:




[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, osijek, 2004."

In [42]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [43]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [44]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [45]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [46]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [47]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [48]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 122, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [49]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.607311
 0.160587
 0.714493
 0.693856
 0.845259
 0.725358
 0.612459
 0.969179
 0.120647
 0.207438

In [50]:
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0        1.0          1.0        …  1.0       1.0          1.0        
 0.607311   0.160587     0.714493      0.969179  0.120647     0.207438   
 0.368827   0.0257881    0.5105        0.939307  0.0145556    0.0430305  
 0.223993   0.00414123   0.364748      0.910356  0.00175608   0.00892617 
 0.136033   0.000665027  0.26061       0.882298  0.000211865  0.00185163 
 0.0826144  0.000106795  0.186204   …  0.855104  2.55608e-5   0.000384098
 0.0501727  1.71498e-5   0.133041      0.828749  3.08382e-6   7.96764e-5 
 0.0304704  2.75403e-6   0.0950571     0.803206  3.72053e-7   1.65279e-5 
 0.018505   4.42261e-7   0.0679176     0.77845   4.48869e-8   3.42852e-6 
 0.0112383  7.10213e-8   0.0485266     0.754457  5.41545e-9   7.11205e-7 

In [51]:
b=rand(n)

10-element Array{Float64,1}:
 0.740709 
 0.635757 
 0.434511 
 0.0410912
 0.933363 
 0.0419936
 0.388915 
 0.688959 
 0.913519 
 0.990192 

In [52]:
x=A\b

10-element Array{Float64,1}:
     -1.20831e8
     -2.55682e5
      3.37009e8
     -1.64906e8
      7.92754e5
     -1.92645e8
      1.40568e8
 -25894.8      
  90239.4      
      2.03737e5

In [53]:
cond(A)

8.727631413981882e9

In [54]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
 -1.208308132110681397309551749412551159116691244224550790812878072715993467066749e+08
 -2.556821001821039150785073490755804853226168555667054556807705907161020950659327e+05
  3.370086013682936823083447120187754186046432909490619657254388641202007632773779e+08
 -1.649056086418036393499804525582264910157120177603335005995248151959166030587366e+08
  7.927539300129839799402206076771155036276701460547188720956855623093728521193877e+05
 -1.926448780996507859829761843933474208204814581568959305086546310072215387788094e+08
  1.405675454147319388875055076067426408764858365860927599644299934494545687162966e+08
 -2.58947706548268862112532441256641636640889256576286000179793436565427568685929e+04 
  9.023937253638861061689066343261740030865130642914046089801138194832966562444675e+04
  2.03737478493676684636532562048449316505659606338496526990948895197098884737959e+05 

In [55]:
norm(xb-x)/norm(xb)

5.652350705763762643556379575170409479165859073632640749492837177735901035825800e-09

### Umjetno loša kondicija

In [56]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)

([-1.0,2.0],50000.00017991671,[8.881784197001252e-16,-4.440892098500626e-16])

## Rezidual

(Prema [G. W. Stewart, Afternotes on Numerical Analysis, str. 128.][Ste96])

Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava

$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

_Rezidual_ (ili _ostatak_) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$
pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,
> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$
vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$
Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphi, 1996"

In [57]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [58]:
norm(r)/(norm(A)*norm(x))

0.0